In [1]:
import sys
import os
import torch 
from pathlib import Path

def get_project_info() -> Path:
  current = Path.cwd().resolve()
  root = current
  for parent in [current, *current.parents]:
    if (parent / "toy_transformers").exists():
      root = parent
      break
  return root, current

if 'ROOT_DIR' not in globals():
	ROOT_DIR, EXPERIMENT_DIR = get_project_info()
	if str(ROOT_DIR) not in sys.path:
		sys.path.append(str(ROOT_DIR))
	if Path.cwd() != ROOT_DIR:
		os.chdir(ROOT_DIR)

from toy_transformers.models import gptv1
from toy_transformers import tokenization

In [2]:
VOCAB_SIZE = 256
BATCH_SIZE = 16
MODE = tokenization.TokenizationMode.STR
DEVICE = "mps"

config = gptv1.GPTv1Config(
	vocab_size=VOCAB_SIZE,
	block_size=256,
)

In [3]:
vocab_path = EXPERIMENT_DIR / f"vocab_{VOCAB_SIZE}.json"
raw_data_path = ROOT_DIR / "data/gutenberg/freud-interpretation-of-dreams.txt"

if not vocab_path.exists():
	raw_data = open(raw_data_path, "r")
	vocab = tokenization.create_bpe(
		raw_data, 
		VOCAB_SIZE, MODE
	)
	vocab.save(vocab_path)
else:
	vocab = tokenization.Vocabulary.load(vocab_path)

In [4]:
data = torch.tensor(
	vocab.encode(open(raw_data_path, "r").read()),
	dtype=torch.long
).to(device=DEVICE)

n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

block_size, batch_size = config.block_size, BATCH_SIZE
def get_batch(split):
  data = train_data if split == 'train' else val_data
  idxs = torch.randint(len(data) - block_size, (batch_size,), device=DEVICE)
  x = torch.stack([data[i:i+block_size] for i in idxs])
  y = torch.stack([data[i+1:i+block_size+1] for i in idxs])
  return x, y

@torch.no_grad()
def estimate_val_loss(model):
  model.eval()
  X, Y = get_batch("val")
  _, loss = model(X, Y)
  model.train()
  return loss.item()

In [5]:
torch.set_float32_matmul_precision("medium")
m = gptv1.LanguageModel(config).to(device=DEVICE)
m.compile()

optimizer = torch.optim.AdamW(m.parameters(), lr=3e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
  optimizer,
  mode='min',
  factor=0.1,
  patience=10
)

import time
from torch.amp import autocast
from tqdm import tqdm

In [6]:
for steps in range(4000):
	xb, yb = get_batch('train')
	with autocast(device_type=DEVICE, dtype=torch.float16):
		logits, loss = m(xb, yb)
	optimizer.zero_grad(set_to_none=True)
	loss.backward()
	optimizer.step()
	train_loss, val_loss = loss.item(), None
	if steps % 50 == 0:
		val_loss = estimate_val_loss(m)
		scheduler.step(val_loss)
	
	if steps % 25 == 0:
		print(steps, train_loss, val_loss)

W0217 19:07:53.402000 1495 torch/_inductor/utils.py:1436] [0/0] Not enough SMs to use max_autotune_gemm mode


InductorError: SyntaxError: failed to compile 
    #include <c10/metal/utils.h>
    #include <c10/metal/reduction_utils.h>
    [[max_total_threads_per_threadgroup(288)]]
    kernel void generated_kernel(
        device float* out_ptr2,
        device float* out_ptr5,
        device half* out_ptr6,
        constant half* in_ptr0,
        constant half* in_ptr1,
        constant half* in_ptr2,
        constant half* in_ptr3,
        constant half* in_ptr4,
        constant half* in_ptr5,
        constant half* in_ptr6,
        constant half* in_ptr7,
        constant half* in_ptr8,
        constant half* in_ptr9,
        constant half* in_ptr10,
        constant half* in_ptr11,
        constant half* in_ptr12,
        constant half* in_ptr13,
        constant half* in_ptr14,
        constant half* in_ptr15,
        constant half* in_ptr16,
        constant half* in_ptr17,
        constant half* in_ptr18,
        constant half* in_ptr19,
        constant half* in_ptr20,
        constant h

In [ ]:
idx = torch.tensor([vocab.encode("The mind ")], dtype=torch.long, device=DEVICE)
print(idx)
print("The mind ", end="", flush=True)
for token in m.generate(idx, max_new_tokens=200):
	print(vocab.decode([token.item()])[0], end="", flush=True)
print()

tensor([[ 36, 126, 145, 129,  47]], device='mps:0')
The mind upill Inauentally noses which arawary noking a clf acou reccrfLef are implapodreams po with repecturness

Iftersompar

KeyboardInterrupt: 